# Praca domowa nr 3

Adrianna Grudzień\
29 marca 2021 r. 

W ramach pracy domowej z przedmiotu Warsztaty badawcze zapoznanałam się z pojęciem `Ablation study`.

# `Ablation study` - badanie ablacji

Badanie ablacji bada wydajność systemu sztucznej inteligencji poprzez usunięcie niektórych komponentów, aby zrozumieć wkład komponentu w cały system. Termin ten jest przez analogię do biologii (usuwanie składników organizmu) i kontynuując analogię, jest szczególnie używany w analizie sztucznych sieci neuronowych, analogicznie do ablacyjnej chirurgii mózgu. Badania ablacji wymagają, aby system wykazywał wdzięczną degradację: aby nadal funkcjonował, nawet gdy brakuje pewnych składników lub są one zdegradowane.

# Przykład

In [185]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

from keras.utils import to_categorical
from keras import models, layers
from keras.datasets import imdb
from sklearn.metrics import classification_report, precision_score


MAX_NUM_WORDS = 20
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=MAX_NUM_WORDS)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

# Funkcja potrzebna, żeby w każdej recenzji było tyle samo liczb
# Tam, gdzie jest ich mniej niż MAX_NUM_WORDS, uzupełniamy zerami
# W pozostałych przypadkach zliczamy liczbę użytych słów.
# ~ coś w rodzaju one hot encoding
def wektoryzacja(data, rozmiar=MAX_NUM_WORDS):
    output = np.zeros((len(data), rozmiar))
    for i, seq in enumerate(data):
        for j in seq:
            output[i,j] += 1
    return output
# Wektoryzacja 
def wektoryzacja_2(data, rozmiar=MAX_NUM_WORDS):
    output = np.zeros((len(data), rozmiar))
    for i, seq in enumerate(data):
        output[i,seq] = 1
    return output

data = wektoryzacja(data)
targets = wektoryzacja_2(targets,2)
targets = np.array(targets).astype('float32')

X_train = data[MAX_NUM_WORDS:]
y_train = targets[MAX_NUM_WORDS:]
X_test = data[:MAX_NUM_WORDS]
y_test = targets[:MAX_NUM_WORDS]

In [186]:
# Możemy przyjrzeć się zmiennej docelowej:
targets

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [182]:
# def wykres(results):
#     plt.figure(figsize=(15,5))
#     # summarize history for accuracy
#     plt.subplot(1, 2, 1)
#     plt.plot(results.history['accuracy'])
#     plt.plot(results.history['val_accuracy'])
#     plt.title('Model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['acc ~ train', 'val_acc ~ test'], loc='upper left')
#     # summarize history for loss
#     plt.subplot(1, 2, 2)
#     plt.plot(results.history['loss'])
#     plt.plot(results.history['val_loss'])
#     plt.title('Model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

def czas_treningu(X_test, y_test, model):
    start = time.time()
    results = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test), verbose=0)
    end = time.time()
    print("Czas treningu: ", end-start, 's')
    return results

In [177]:
# funkcja do opracowania wyniku:
def analiza_modelu(X_test, y_test, model):
    results = czas_treningu(X_test, y_test, model)
    score = model.evaluate(X_test, y_test, verbose=0)
    print('loss: ',score[0])
    print('accuracy: ',score[1])

### Budowa modelu

In [187]:
model = models.Sequential()
# Input layer
model.add(layers.Dense(50, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.5, noise_shape=None, seed=None))

# Output layer
model.add(layers.Dense(2, activation='softmax'))

model.summary()

# kompilujemy model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# trenujemy model
analiza_modelu(X_test, y_test, model)

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_307 (Dense)            (None, 50)                1050      
_________________________________________________________________
dropout_202 (Dropout)        (None, 50)                0         
_________________________________________________________________
dense_308 (Dense)            (None, 50)                2550      
_________________________________________________________________
dropout_203 (Dropout)        (None, 50)                0         
_________________________________________________________________
dense_309 (Dense)            (None, 50)                2550      
_________________________________________________________________
dropout_204 (Dropout)        (None, 50)                0         
_________________________________________________________________
dense_310 (Dense)            (None, 2)               

# Dodawanie/odejmowanie `warstw`

In [171]:
# dodanie warstw

model = models.Sequential()
# Input layer
model.add(layers.Dense(50, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
# Output layer
model.add(layers.Dense(2, activation='softmax'))

# model.summary()

# kompilujemy model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

Czas treningu:  139.21098279953003 s
loss:  0.6209396123886108
accuracy:  0.550000011920929


In [174]:
# usunięcie warstw Dropout

model = models.Sequential()
# Input layer
model.add(layers.Dense(50, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))

# Output layer
model.add(layers.Dense(2, activation='softmax'))

# model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

Czas treningu:  95.57809782028198 s
loss:  0.5147997736930847
accuracy:  0.699999988079071


Wychodzi na to, że w przypadku tych danych, im więcej warstw, tym accuracy gorsze i im mniej warstw, tym accuracy większe.

# Zmiana parametrów `warstw`

Zamiast softmax jest sigmoid.

In [178]:
model = models.Sequential()
# Input layer
model.add(layers.Dense(50, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.5, noise_shape=None, seed=None))

# Output layer
model.add(layers.Dense(2, activation='sigmoid'))

# model.summary()

# kompilujemy model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# trenujemy model
analiza_modelu(X_test, y_test, model)

Czas treningu:  164.43071794509888 s
loss:  0.6727690100669861
accuracy:  0.6000000238418579


Model trenuje się dłużej, loss jest większy, ale accuracy nie zmienia się.

### W warstwach Dense: zamiast 50 - 100:

In [180]:
model = models.Sequential()
# Input layer
model.add(layers.Dense(100, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.5, noise_shape=None, seed=None))

# Output layer
model.add(layers.Dense(2, activation='softmax'))

# model.summary()

# kompilujemy model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# trenujemy model
analiza_modelu(X_test, y_test, model)

Czas treningu:  196.11490750312805 s
loss:  0.5822020173072815
accuracy:  0.6000000238418579


Model trenuje się ponad dwa razy dłużej, loss jest mniejszy, accuracy - bez zmian.

### Zmiana parametrów `treningu`

In [181]:
odel = models.Sequential()
# Input layer
model.add(layers.Dense(50, activation='relu', input_shape=(MAX_NUM_WORDS, )))
# Hidden layers:
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.5, noise_shape=None, seed=None))

# Output layer
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
analiza_modelu(X_test, y_test, model)

Czas treningu:  246.3105251789093 s
loss:  0.6067681312561035
accuracy:  0.6499999761581421
Czas treningu:  211.8729703426361 s
loss:  0.6000439524650574
accuracy:  0.6499999761581421
Czas treningu:  211.44419050216675 s
loss:  0.5919212102890015
accuracy:  0.6000000238418579
Czas treningu:  197.89110898971558 s
loss:  0.6036732196807861
accuracy:  0.699999988079071


Jak widać, dla różnych parametrów dostajemy różne wyniki. Jeśli w tym przypadku zależy nam na czasie, to najlepiej będzie wybrać optimizer='sgd', ale nie zawsze będzie to odpowiednie.